# Lab 10: Exploring paleoclimate variability with a transient climate model

The goal of this lab is to build on the skills for working with climate data in netCDF format that we've been developing over the course of the semester. This time we will explore past climate variability using ice sheet reconstructions and output from a transient climate model - the TraCE-21ka simulation. 

TraCE-21ka output comes from a fully-coupled, non-accelerated atmosphere-ocean-sea ice-land surface CCSM3 simulation that simulates the global climate system since the Last Glacial Maximum (LGM) (22ka). More information about TraCE-21ka can be found [here](https://www.earthsystemgrid.org/project/trace.html). This [FAQ page](https://trace-21k.nelson.wisc.edu/faq.html) is also helpful.

Our science questions are:
- How did temperature differ at the LGM relative to the preindustrial and how did it change through tiime?
- How have ice sheets changed since the LGM?
- What factors explain the patterns of climate change over the last 22 ka?

---
## Part 1. Using data from multiple sources to explore climate variability since the LGM
This lab will draw on several processing methods we have used before and multiple different sources of data. Let's start by accessing the datasets that we'll be using. All of the datasets are available in this Google Drive folder: https://drive.google.com/drive/folders/1dv663_F7lVw6C8hgET9xXiR5ayK2s3gx?usp=sharing

Next we will read in global surface temperature data from the TraCE-21ka simulation.

---
### 1. Initial analysis - Global temperature from TraCE-21ka simulation

In [ ]:
# Import the tools we are going to need today:
import matplotlib.pyplot as plt  # plotting library
import pandas as pd
import numpy as np  # numerical library
import xarray as xr  # netCDF library
#!pip install cartopy
import cartopy  # Map projections library
import cartopy.crs as ccrs  # Projections list
!pip install h5netcdf
import h5netcdf
!pip install netcdf4
import netCDF4
# Some defaults:
plt.rcParams['figure.figsize'] = (12, 5)  # Default plot size

We will be making more maps with matplotlib today. If you're curious, the list of color palettes available can be shown by running the cell below.

In [ ]:
from matplotlib import colormaps
list(colormaps)

Read the surface temperature data in and store the variable 'TS' from the netCDF as a new variable. Then inspect it. 

In [ ]:
Model_TS = xr.open_dataset('TraCE-21K-II.ann.TS.nc')
Model_TS = Model_TS.TS
Model_TS

We can use .head(n) and .tail(n) to inspect either the first n entries or last n entries a dataframe.

In [ ]:
print(Model_TS.time.tail(10))

The temporal resolution of this model is decadal and the units are ka BP. This means we have output for every ten years from 22 ka BP to 0.04 ka BP. In the model, a time of 0 ka BP (or -9.900212e-07 ka BP specifically) corresponds to the year 1950. A time of -22 ka BP corresponds to 22,000 years ago. A time of 0.04 corresponds to 0.04 thousand years after 1950, which is 1990.

Let's try plotting the data, starting with a time series of global average temperature throughout the entirety of the simulation.

In [ ]:
ax = plt.axes()
ax.plot(Model_TS.time, Model_TS.mean(dim = ['lat', 'lon']))
ax.set_xlabel('Time (ka BP)'); ax.set_ylabel('Temperature (K)')

Describe the trend that you see.

Before we continue exploring the data from the climate model simulation, head [here](https://www.earthsystemgrid.org/project/trace.html) to learn more about TraCE-21ka. In a Markdown cell below, describe how this simulation was set up. What are the forcings and boundary conditions?

---
### 2. Calculating and plotting anomalies 

When working with model data, as with observational or reanalysis data, it is often helpful to calculate anomalies between different time intervals of the simulation. Let's calculate an anomaly for the LGM (-22 ka BP) relative to the preindustrial (PI). The preindustrial refers to the most recent decades before anthropogenic warming set in, usually considered ~1750 - 1850. This is a useful reference point for comparison with paleoclimate variability.

In [ ]:
PI_time_slices = np.arange(-0.19, -0.09, 0.01) # define the time slices for the PI (remember 0 ka BP is 1950)
# can we tell what np.arange is doing for us here?

# make a new variable with surface temperatures for for these time slices
Model_TS_PI = Model_TS.sel(time = PI_time_slices, method = 'nearest')
Model_TS_PI # inspect it

Note that we can include 'method = 'nearest' when we use .sel to have Python select the times that are closest to those in our PI_time_slices array. This is handy because it means we don't have to match the times in the model dataframe exactly.

Let's check to make sure that we have isolated the PI correctly. We don't want to include the anthropogenic warming that has occurred since 1850. The code cell below will plot a time series of the final 30 decades of the simulation and our new PI variable.

In [ ]:
ax = plt.axes()
ax.plot(Model_TS.time.tail(30),Model_TS.mean(dim = ['lat', 'lon']).tail(30), label = 'Last 300 years')
ax.plot(Model_TS_PI.time,Model_TS_PI.mean(dim = ['lat', 'lon']), color = 'red', label = 'PI')
ax.legend(loc = 'best')
ax.set_xlabel('Time (ka BP)'); ax.set_ylabel('Temperature (K)')

Now let's isolate the first 100 years of the simulation, corresponding the the LGM.

In [ ]:
LGM_time_slices = np.arange(-22, -21.901, 0.01)
Model_TS_LGM_mean = Model_TS.sel(time = LGM_time_slices, method = 'nearest').mean(dim = 'time')

Check the dimensions of the dataframe that we produced. Is this what we need in order to make a map?

If so, let's start by making a map of global temperatures at the LGM.

In [ ]:
ax = plt.axes(projection=ccrs.Robinson())
Model_TS_LGM_mean.plot(ax=ax, transform=ccrs.PlateCarree(), vmin=210, vmax=300)
ax.coastlines(); ax.gridlines(); ax.set_title('LGM')

#### *Exercise:* 
Now try calculating the global temperature anomaly between the LGM (22 ka BP) and the PI (1750 - 1850).

Then make a similar map as above, but showing global temperatures during the PI. 

Then create a map showing the difference between the LGM and the PI. 

How much colder was the LGM on average relative to the PI?

---
### 3. Exploring ice sheet reconstructions

The LGM was characterized by large ice sheets in the northern Hemisphere. Let's take a look at some reconstructions of these ice sheets through time. The netCDF file called 'reconstruction_a1_1_degree.nc' has variables for ice sheet thickness and sea level at 2,500 year intervals for the past 80,000 years. We are going to focus on 22.5 ka BP onward.

Read in the ice reconstruction data and trim it down to just the last 22,500 years so that it spans the same range as our TraCE-21ka simulation.

In [ ]:
Ice_recon = xr.open_dataset('reconstruction_a1_1_degree.nc', decode_times=False)
Deglacial_time_slices = np.arange(-22500, 10, 2500)
Ice_recon = Ice_recon.sel(time = Deglacial_time_slices, method = 'nearest')

Try making a map the reconstructed ice thickness at the year -22,500 BP. You'll need to use .sel to pull out the spatial data for one specific time slice.

In [ ]:
# Your plotting script here

Now let's overlay the reconstructed ice thickness on our plot of global temperature at the LGM. To do this, we need to set every grid cell that doesn't have any ice in it to NaN so that it doesn't totally block our underlying temperature data.

In [ ]:
Ice_recon_masked = Ice_recon.where(Ice_recon.ice_thickness > 0)

Inspect Ice_recon_masked to see how it differs from Ice_recon.

Now create a map with two layers: global temperature at the LGM and the thickness of the ice sheets at the LGM.

---
### 4. Incorporating CO2 and insolation data

Varying atmospheric CO2 levels and changes in the distribution of incoming solar radiation are key aspects of climate variability since the LGM. Read in the .csv files that hold insolation and CO2 data using pd.read_csv(file path). We'll use these for the Lab Procedure.

In [ ]:
# Your code to read in the .csv files here.

Take a close look at the insolation data and try plotting it with the script below. I get you started with a plot of '90NJune'. Add each of the different columns to the plot. In a Markdown cell describe how far back in time this dataset goes? What is the difference between the different columns? If we're interested in the role insolation plays in the growth and retreat of northern hemisphere ice sheets, which column are we going to be most interested in? 

In [ ]:
ax = plt.axes()
ax.plot(insolation['Kiloyear before present'], insolation['90NJune'], label = '90NJune')
ax.legend(loc = 'best')

---
## Part 2: Lab Procedure

1. Choose a new time span of at least 100 years from the model simulation. Calculate the mean temperature during this time and the anomaly between this time and the PI. 
    - Create a time series showing the full global average temperature record and highlight your new interval.
    - Create a map of the temperature anomaly between your innterval and the PI and include the configuration of the ice sheets at the nearest point in time for which ice data is available.
 <br/><br/>
2. Create a time series plot that shows modelled temperature variability through time for the northern hemisphere and for the southern hemisphere. How does temperature change through time differ between the two hemispheres?
 <br/><br/>
3. Create one figure with multiple subplots that show:
    - the insolation curve since the LGM
    - atmsopheric CO2 concentration
    - average sea level since the LGM
    - the average thickness of ice on Earth since the LGM
    - the average global temperature since the LGM
<br/>
Note that each different type of data has a different way of describing time. Consider how you can adjust the time components so that they all plot nicely on one graph.
 <br/><br/>
4. At the end of the day, all climate models are simplified simulations of the climate system. How could we assess whether this simulation is a useful representation of changes to the climate system through time?

---
## Endnotes
- The TraCE-21ka data are available [here](https://trace-21k.nelson.wisc.edu/portal.html).
- The ice sheet reconstruction is from [Gowan et al. 2021](https://www.nature.com/articles/s41467-021-21469-w).
- Insolation data are available [here](https://www.ncei.noaa.gov/access/paleo-search/study/5776)
- CO2 data are available [here](https://www.usap-dc.org/view/dataset/609108)
- Last update: 23 Apr. 2025, Cam de Wet